In [ ]:
def source_out(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

def target_out(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

In [ ]:

num_batches = len(data_loader)

In [ ]:
# Discriminator and Generator
discriminator_GAN = Discriminator()
generator_GAN = Generator()
if torch.cuda.is_available():
    discriminator_GAN.cuda()
    generator_GAN.cuda()

In [ ]:

# Optimizers
d_optimizer_GAN = optim.Adam(discriminator_GAN.parameters(), lr=0.0002)
g_optimizer_GAN = optim.Adam(generator_GAN.parameters(), lr=0.0002)

# Loss function
loss = nn.BCELoss()

# Number of steps to apply to the discriminator
d_steps = 1  # In Goodfellow et. al 2014 this variable is assigned to 1
# Number of epochs
num_epochs = 200

In [ ]:
def train_discriminator(optimizer, trans_source_data, trans_target_data):
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on source Data
    prediction_source = discriminator_GAN(trans_source_data)
    # Calculate error and backpropagate
    error_source = loss(prediction_source, source_out(trans_source_data.size(0)))
    error_source.backward()

    # 1.2 Train on Fake Data
    prediction_target = discriminator_GAN(trans_target_data)
    # Calculate error and backpropagate
    error_target = loss(prediction_target, target_out(trans_target_data.size(0)))
    error_target.backward()
    
    # 1.3 Update weights with gradients
    optimizer.step()
    
    # Return error
    return error_source + error_target, prediction_source, prediction_target


In [ ]:
def train_generator(optimizer, target_data):
    # 2. Train Generator
    # Reset gradients
    optimizer.zero_grad()
    # Sample noise and generate fake data
    prediction = discriminator_GAN(fake_data)
    # Calculate error and backpropagate
    error = loss(prediction, source_out(prediction.size(0)))
    error.backward()
    # Update weights with gradients
    optimizer.step()
    # Return error
    return error

In [ ]:
logger = Logger(model_name='Time-Series-GAN', data_name='Time-Series-Data')

for epoch in range(num_epochs):
    for n_batch, (real_batch,_) in enumerate(data_loader):

        # 1. Train Discriminator
        trans_source_data = Variable(images_to_vectors(real_batch)) ### ? get from the previous transformer
        if torch.cuda.is_available(): real_data = real_data.cuda()
        # Generate fake data
        trans_target_data = generator_GAN().detach() ## ? from generator get trans_target_data
        # Train D
        d_error, d_pred_source, d_pred_target = train_discriminator(d_optimizer_GAN,
                                                                trans_source_data, trans_target_data)

        # 2. Train Generator
        # Generate fake data
        target_data = generator_GAN(??) # get data from generator transformer
        # Train G
        g_error = train_generator(g_optimizer_GAN, trans_target_data)
        # Log error
        logger.log(d_error, g_error, epoch, n_batch, num_batches)

        # Display Progress
        if (n_batch) % 100 == 0:
            display.clear_output(True)
            # Display Images
            # test_images = vectors_to_images(generator(test_noise)).data.cpu()
            # logger.log_images(test_images, num_test_samples, epoch, n_batch, num_batches);
            # Display status Logs
            logger.display_status(
                epoch, num_epochs, n_batch, num_batches,
                d_error, g_error, d_pred_source, d_pred_target
            )
        # Model Checkpoints
        logger.save_models(generator_GAN, discriminator_GAN, epoch)